<h1>
    Import Modules

In [ ]:
import string
import os
import pickle as pkl
from os import walk
import pandas as pd
import numpy as np
import csv
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import tensorflow
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from livelossplot.tf_keras import PlotLossesCallback
import nltk
nltk.download('punkt')
import unicodedata
import joblib
from tensorflow import keras
import pickle

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from data_wrangling import data_wrangling
from is_diacritic import is_diacritic
from is_char import is_char
from split_daicrtics_from_arabic_text import split_daicrtics_from_arabic_text
from padding_sequence import padding_sequence
from concate_arabic_letters_astext_with_daicritic import concate_arabic_letters_astext_with_daicritic
from concate_arabic_letters_asnumber_with_daicritic import concate_arabic_letters_asnumber_with_daicritic
from remove_diacritics import remove_diacritics

In [ ]:
ARABIC_LETTERS = frozenset([chr(x) for x in (list(range(0x0621, 0x63B)) + list(range(0x0641, 0x064B)))])
diacritic = ['َ', 'ً', 'ُ', 'ٌ', 'ِ', 'ٍ', 'ْ', 'ّ', 'َّ', 'ًّ', 'ُّ', 'ٌّ', 'ِّ', 'ٍّ','']

In [ ]:
CHAR2INDEX = dict((l, n) for n, l in enumerate(sorted(ARABIC_LETTERS)))
CHAR2INDEX.update(dict((v, k) for k, v in enumerate([' '], len(CHAR2INDEX))))

In [ ]:
DIAC2INDEX= {char: i for i, char in enumerate(diacritic)}

<h1>
    Loading Tashkeela Corpus (Cleaned Version) from Huging Face

In [ ]:
import datasets

<h1>
    Loading Train data and Convert it into DataFrame

In [ ]:
train = datasets.load_dataset('arbml/tashkeelav2', split='train')


In [ ]:
train = pd.DataFrame(train)


In [ ]:
train.head()

,diacratized,text
0,إلَى مُوَكَّلِي الدَّائِنِ فَفِي حِكَايَتِهِ ه...,إلى موكلي الدائن ففي حكايته هذه إلزام الموكل ا...
1,فَإِنْ أَبْرَءُوهُ بَعْدَ الْوَكَالَةِ لَمْ يَ...,فإن أبرءوه بعد الوكالة لم يكن وكيلا في قبضه أي...
2,لِأَقْسَامٍ مِنْ الْعَرَبِيَّةِ جَعَلُوهَا أَي...,لأقسام من العربية جعلوها أي علماء هذا العلم ال...
3,بِالْأَشْيَاءِ كَوْنُهَا وَيُسَمَّى هَذَا عِلْ...,بالأشياء كونها ويسمى هذا علما بما سيكون ثم يعل...
4,فَمَنْ تَعَجَّلَ فِي يَوْمَيْنِ فَلَا إِثْمَ ع...,فمن تعجل في يومين فلا إثم عليه ومن تأخر فلا إث...


In [ ]:
train.tail()

,diacratized,text
522458,وَلَنَا مَا رُوِيَ أَنَّ النَّبِيَّ صَلَّى الل...,ولنا ما روي أن النبي صلى الله عليه وسلم صلى عل...
522459,وَهِيَ أَيْ صَدَقَةُ الْفِطْرِ نِصْفُ صَاعٍ مِ...,وهي أي صدقة الفطر نصف صاع من بر أو دقيقه أو سو...
522460,قَوْلُهُ الْمُقْتَضَى أَوْ الْمُخَيَّرُ الْإِس...,قوله المقتضى أو المخير الإسناد فيهما مجازي إذ ...
522461,طَلَبَهُ وَامْتَنَعَ الْمُشْتَرِي مِنْهُ لِأَن...,طلبه وامتنع المشتري منه لأن المشتري قد يكون غر...
522462,قُلْتُ فَإِنْ كَانَ أَرَادَ أَنْ يَرُدَّ قَالَ...,قلت فإن كان أراد أن يرد قال مالك ذلك له فإن قا...


In [ ]:
# Checking Length of diacratized text column in dataFrame
diacritic_train = train['diacratized']
len(diacritic_train)

522463

<h1>
    Loading Test data & Convert it into DataFrame

In [ ]:
test = datasets.load_dataset('arbml/tashkeelav2', split='test')

In [ ]:
test = pd.DataFrame(test)

In [ ]:
test.head()

,diacratized,text
0,قَالَ وَإِنْ كَانَ فِيهَا شَيْءٌ بَارٌّ لَا يَ...,قال وإن كان فيها شيء بار لا يجوز فإنه حانث قلت...
1,قِيَامِ الْمُسْتَحِقِّ فَقَالَ ابْنُ الْمَوَّا...,قيام المستحق فقال ابن المواز لا شيء على المشتر...
2,فَقَدْ اجْتَهَدَ الْإِمَامُ الشَّافِعِيُّ وَال...,فقد اجتهد الإمام الشافعي والإمام محمد والإمام ...
3,فِيمَنْ أَنْفَقَ عَلَى ضَالَّةٍ وَفِي الْوَصِي...,فيمن أنفق على ضالة وفي الوصي يرهن لليتيم رهنا ...
4,أَخْبَرَنَا أَبُو عَبْدِ اللَّهِ الْحَافِظُ أَ...,أخبرنا أبو عبد الله الحافظ أخبرنى أحمد بن سهل ...


In [ ]:
test.tail()

,diacratized,text
58047,أَنَّ مَا نَقَلَهُ الشَّارِحُ أَوَّلًا عَنْ ال...,أن ما نقله الشارح أولا عن الفتح من أن الصريح م...
58048,مَنْ يَدْخُلُ الْجَنَّةَ يَوْمَ الْقِيَامَةِ و...,من يدخل الجنة يوم القيامة ولا فخر وإني آتي باب...
58049,فَإِنْ احْتَجَمَ فَظَنَّ أَنَّ ذَلِكَ يُفْطِرُ...,فإن احتجم فظن أن ذلك يفطره فأكل بعد ذلك متعمدا...
58050,الْقَاعِدَتَيْنِ وَذَلِكَ أَنَّ الْقَائِلَ الْ...,القاعدتين وذلك أن القائل الأول يقول غروب الشمس...
58051,الْآيَةُ الْخَامِسَةُ وَالسِّتُّونَ قَوْله تَع...,الآية الخامسة والستون قوله تعالى للذين يؤلون م...


In [ ]:
# Checking Length of diacratized text column in dataFrame
diacritic_test = test['diacratized']
len(diacritic_test)

58052

# Data Cleaning & Preprocessing

# Wrangling and Retrieve Cleaned Train DataFrame

In [ ]:
train['diacratized']  = train['diacratized'].apply(data_wrangling)

# Wrangling and Retrieve Cleaned Test DataFrame

In [ ]:
test['diacratized']  = test['diacratized'].apply(data_wrangling)

# Taking A Part of DataFrame

In [ ]:
train_part = train.iloc[:60000, :]
test_part  = test.iloc[:6000,:]

# Taking only the first 15 words from sentences

In [ ]:
train_part['diacratized'] = train_part['diacratized'].str.split().apply(lambda x: ' '.join(x[:15]))
test_part['diacratized'] = test_part['diacratized'].str.split().apply(lambda x: ' '.join(x[:15]))

# Applying Split Daicrtics From Arabictext

In [ ]:
train_part[['X' ,'y']] = train_part['diacratized'].apply(lambda x: pd.Series(split_daicrtics_from_arabic_text(x)))
test_part[['X' ,'y']] = test_part['diacratized'].apply(lambda x: pd.Series(split_daicrtics_from_arabic_text(x)))

# Applaying Padding

In [ ]:
max_length = 100

#padding train part
X , y = padding_sequence(max_length , train_part['X'] ,len(CHAR2INDEX)-1, train_part['y'] ,len(diacritic)-1 )

#padding test part
X_test , y_test = padding_sequence(max_length , test_part['X'] ,len(CHAR2INDEX)-1, test_part['y'] ,len(diacritic)-1)

# Applaying Train Validation Split

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(X, y, test_size=0.04, random_state=42)

# Concate arabic letters with it's correct daicritics

In [ ]:
# Apply concate_arabic_letters_astext_with_daicritic on Validation Data
all_val = []
for i in range(len(x_val)):
    concat_str = concate_arabic_letters_asnumber_with_daicritic(x_val[i], y_val[i], max_length)
    all_val.append(concat_str)

In [ ]:
# Apply concate_arabic_letters_astext_with_daicritic on Test Data
all_test = []
for i in range (len(X_test)):
    concat_str = concate_arabic_letters_asnumber_with_daicritic(X_test[i] , y_test[i] , max_length )
    all_test.append(concat_str)

# Saving Train and Test

In [ ]:
# Specify the directory path to save the files
save_directory = '/content/drive/MyDrive/Utils'

# Create the directory if it doesn't exist
os.makedirs(save_directory, exist_ok=True)

# Save x_train
with open(os.path.join(save_directory, 'X_train.pkl'), 'wb') as f:
    pickle.dump(x_train, f)

# Save y_train
with open(os.path.join(save_directory, 'y_train.pkl'), 'wb') as f:
    pickle.dump(y_train, f)

# Save x_test
with open(os.path.join(save_directory, 'X_val.pkl'), 'wb') as f:
    pickle.dump(x_val, f)

# Save y_test
with open(os.path.join(save_directory, 'y_val.pkl'), 'wb') as f:
    pickle.dump(y_val, f)

# Save orginal_validation data
with open(os.path.join(save_directory,'val_orginal.txt'), 'w') as f:
    for item in all_val:
        f.write("%s\n" % item)
# Save test data
with open(os.path.join(save_directory,'test_orginal.txt'), 'w') as f:
    for item in all_test:
        f.write("%s\n" % item)